In [32]:
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
from collections import defaultdict


In [33]:
import spacy
spacy.load('en')

In [36]:
def text_cleaner(text):
    nlp = spacy.load('en', tagger=False, parser=False, matcher=False)
    doc = nlp(text)

    out_dict = defaultdict(list)

    out_dict['tokens'] = [t.lemma_ for t in doc if not (t.is_stop or t.pos_ == "PUNCT") ]

    for ent in doc.ents:
        out_dict['Named Entity - '+ ent.label_].append(ent.text)

    return out_dict 

text = "Mary had a little lamb called Mary."

out_dict = text_cleaner(text)

print(text)
print(out_dict)

for t in out_dict['tokens']:
    print(t)


Mary had a little lamb called Mary.
defaultdict(<class 'list'>, {'tokens': ['mary', 'little', 'lamb', 'call', 'mary'], 'Named Entity - PERSON': ['Mary', 'Mary']})
mary
little
lamb
call
mary
